# Data Science of High Energy Physics Project Report
## Title: TrackML Particle Tracking Challenge (High Energy Physics particle tracking in CERN detectors)
## Author: Stephan Potgieter (PTGJAK001)

### Introduction
The Large Hadron Collider (LHC) is a high energy particle collider set up by CERN to experimentally investigate predictions and properties of the Standard Model - with the hope to either verify the model or uncover new physics that needs to be accounted for. Experiments like ATLAS and ALICE make use of the LHC to collide certain hadrons (heavy-ions like protons or lead nuclei in the case of ALICE) and then investigate the properties and distributions of the particles that scatter out from these collisions.

Multiple detectors are set up around the collision beam to gather enough information about the scattering particles, with the aim to be able to identify all of them as accurately as possible and obtain enough particle-specific information (like momentum and energy deposit) to investigate statistical distributions of the resulting collisions. On a very simplified level, these detectors make use of certain materials and electronics to measure "hits" of the particles that pass through them. After a collisions takes place, the detectors will "light-up" with many hits from the resulting scattering particles that pass through them. Thus, a very important task is then to be able to use this hit information to identify and fit global tracks of individual particles.

#### Problem Statement:
In this project we will attempt the TrackML Particle Tracking Challenge presented by Kaggle, where we will investigate whether a Neural Network can be meaningfully used to accurately join hits from detectors to form tracks of the particles associated with them.

### Approach and Analysis:
We will approach this challenge by using a simple sequential neural network as the main tool to attempt to infer particle tracks from the many hits seen in the detectors. In order to construct and train the neural network to produce meaningful and usable output, we first need to inspect the given data of many events - containing simulated measurements of particles from a proton-proton collision at the LHC.

#### Data format:
The format of the given data is the same for each indpendent event. The 'hits' file contains the entries of all the hits that were measured during that event, where the format is as follows:

<img src="./images/hits.png" alt="drawing" width="600" align="left"/>
</br></br></br></br></br>

For each hit we see that we are given the global coordinates (x,y,z) and the volume/alyer/module id's of the region of the detector, which refers to the entries of an additional 'detectors' file (same accross all events) that contains geometric information about the detector.

The 'cells' file contains the constituent active detector cells that comprise each hit, where a cell is the smallest granularity inside each detector module. It can have up to 10 cell entries per hit in the 'hits' file, with at least one per hit. It has the following format:

<img src="./images/cells.png" alt="drawing" width="280" align="left"/>
</br></br></br></br></br></br></br></br></br>

The value given for each cell is the signal information of that detector, for example the charge deposition measurement in the case of a gass ionization detector.

Since the data is simulated for specific known particles, there is also truth information for each event. Which simply assigns a unique particle id to each hit in the event. There is also a 'particles' file that contains some information for each particle, like the momentum components and number of hits associated with that particle.

#### Approach to the problem:
The main objective with the given information is to find a way to construct a neural network that can combine/assign the hits to form particle tracks. When training we have access to the truth information that tells us how many particles and tracks we expect, but the end goal is to be able to build up all the required tracks from test data without any notion of the number of particles. Due to a neural network having a static structure w.r.t its input and output dimentions, this places a restriction on how the network can be used. At a simple level we could hope to merely input hits to the network and have it output information about all the tracks, but clearly this isn't possible, as we won't even know how many tracks to look for.

Thus, the chosen approach was to consider pairs of hits and have the NN output a single probability of them being part of the same track. One could just as easily consider hit triplets or quadruplet, but due to an event having on average around 120000 events, the number of possible permutations are too much to computationally handle realistically - a problem we are even posed with for pairs.

#### Input to Neural Network:
For each hit of the pair we input the following:
- x,y,z global coordinates
- Count of cells
- Sum of values of the cells

Then using the two hits, we determine the shortest distance between a straight line fit of the hits and the origin of the detector where the particles originate from. This was only added later in the analysis, as the network was struggling to learn a distinction between the two classes, so an assumption was made that this was due to there not being enough geometric information as input. Here an argument can be made for rather using hit triplets, as the tracks we are attempting to fit with hits will have a helical shape and no notion of curve can be inferred from only two hits.

#### Filtering of hit pairs:
Initially the shortest distance from the line fit to the origin was used as a filter for the hit pairs, as it makes sense to only consider the case where a physical curved track is realistically possible. This lead to issues of unbalanced data sets and the accuracy of the filter was not very reliable, so it was chosen to rather add the quantity as input to the NN and allow it to statistically infer a geometric relationship. In the case of trying hit triplets, we could either try to perfome some helical fit and use the parameters of the fit as input - can discard triplets where fit is not possible.

#### Dataset construction:
From initally using the filtered dataset to train the network, it became clear that the network was not performing well at all with being able to identify true hit pairs. As already mentioned, the filtered hit pairs were very unbalanced w.r.t the two classes, with only around 10% being true hit pairs. From this it was decided to use the truth data to construct a balanced dataset comprising of exactly half of each of the two classes. Since there are so many possible permutations, this approach will only consider a fraction of all the possible pair combinations for each event.

Initially only a single event was used to construct the dataset, but it soon became clear that this lead to the network not performing well for tests from other events. The assumption is that this is due to the network over-learning for the tracks of that single event. So finally the dataset was constructed by combing an equal number of hit pairs (40000) accross 20 events.

The test datasets were constructed in the same manner, by first looking at a single event for two different pair counts, and then accross multiple events for two different sets spanning different ranges of the available data.

#### NN configuration and training:
The configuration and construction of the final network was as follows:

<img src="./images/network.png" alt="drawing" width="750" align="left"/>
</br></br></br></br></br></br></br>

A few different models were tested with a different number of hidden layers and nodes in each layer. It was found that adding more hidden layers than above did not really improve the performance of the network in any noticeable way. Increasing the number of nodes in each layer did lead to slighly better performance accross the tests of events. The use of the 'selu' activation function was seen as a recommendation in the discussion board on Kaggle.

For training, increasing the batch size and decreasing the number of epochs lead to better performance accross events. Initially 8 epochs and a batch size of 100 was used, but this lead to the network often classifying false hit pairs as true, even though the loss on the training set was much lower than using 3 epochs with batch sizes of 500. A possible reason for this could be over-training on the train dataset when using more epochs and updating the network more often. A validation-split was also added to aid with avoiding over-training.

#### Track construction:
Something we have avoided talking about till now is how to reach the final objective of constructing full tracks from only having information about hit pairs. Even though we do not implement it in this project, it should be algorithmically possible to construct full tracks by picking some hit as a starting point and then building up the track by using the NN output to pick consecutive hits. Consecutive tracks then follow by picking starting hits not assigned to the previous tracks. A valid concern is whether we will be able to assign all the hits to a track with the limited fraction of pairs we consider. Ideally, we would like to consider all the possible pairs of a test event and through this construct all the tracks - in this scenario the use of a filter could be necessary.

### Results:



### Conclusion:
